# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nba.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [317]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

'''
CITIES DATAFRAME CLEANUP AND PREPARATION
'''
#Removing trailing * and [xyz] from all columns
cities.replace(to_replace = "\[.*\]$", value ="", inplace = True, regex=True)
cities.replace(to_replace = ".*—$", value ="", inplace = True, regex=True)
cities.replace(to_replace = ".*—\s$", value ="", inplace = True, regex=True)

#Replacing blank/epmty spaces with np.NaN
cities.replace(to_replace = r'^\s*$', value =np.nan, inplace = True, regex=True)

#Sorting for checking names across cities and sports dataframes
cities.sort_values(by='Metropolitan area', inplace=True)

#Dropping rows with np.NaN in the NHL column since we are analysis only NHL for this question
cities.dropna(subset=['NHL'], inplace=True)

#Joining NHL column with cities dataframe
cities["team"] = cities[["Metropolitan area","NHL"]].apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)

#Reducing the size of the cities dataframe after renaming Population (2016 est.)[8] column
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities = cities[['Metropolitan area','Population', 'team']]

#astype of all DFs
cities["Population"] = cities["Population"].astype(float)
cities['team'] = cities['team'].astype(str)

cities

'''
NHL DATAFRAME CLEANUP AND PREPARATION
'''
#Subsetting NHL dataframe tro 2018 only
nhl_df_2018 = nhl_df[nhl_df["year"] == 2018]

#Strips white spaces
nhl_df_2018['team'] = nhl_df_2018['team'].str.strip()
cities['team'] = cities['team'].str.strip()

#Cleaning
nhl_df_2018['team'].replace(to_replace = "[(].*", value ="", inplace = True, regex=True)
nhl_df_2018['team'].replace(to_replace = "[\*]", value ="", inplace = True, regex=True)
nhl_df_2018['team'].replace(to_replace = "[+]$", value ="", inplace = True, regex=True)

#Making Win and Loss columns numeric and dropping strings or nan
nhl_df_2018['W'] = pd.to_numeric(nhl_df_2018['W'],errors='coerce')
nhl_df_2018.dropna(inplace=True)
nhl_df_2018['L'] = pd.to_numeric(nhl_df_2018['L'],errors='coerce')

#dictionary to fix names in the respective sports sheet
names_fix = {"Minnesota":"Minneapolis–Saint Paul",
             "Florida":"Miami–Fort Lauderdale",
             "Dallas":"Dallas–Fort Worth",
             "Carolina Hurricanes":"Raleigh Hurricanes",
             "Colorado":"Denver",
             "Vegas":"Las Vegas",
             "Los Angeles Kings":"Los Angeles KingsDucks",
             "Anaheim Ducks":"Los Angeles KingsDucks",
             "New York Rangers":"New York City RangersIslandersDevils",
             "Arizona":"Phoenix",
             "San Jose":"San Francisco Bay Area",
             "Tampa Bay":"Tampa Bay Area",
             "Washington":"Washington, D.C.",
             "Los Angeles Clippers":"Los Angeles LakersClippers",
             "Los Angeles Lakers":"Los Angeles LakersClippers",
             "Golden State Warriors":"San Francisco Bay Area Warriors",
             "Miami":"Miami–Fort Lauderdale",
             "New York Knicks":"New York City KnicksNets",
             "Utah":"Salt Lake City",
             "Brooklyn Nets":"New York City KnicksNets",
             "Chicago Cubs":"Chicago CubsWhite Sox",
             "Chicago White Sox":"Chicago CubsWhite Sox",
             "Texas Rangers":"Dallas–Fort Worth Rangers",
             "Los Angeles Angels":"Los Angeles DodgersAngels",
             "Los Angeles Dodgers":"Los Angeles DodgersAngels",
             "New York Mets":"New York City YankeesMets",
             "New York Yankees":"New York City YankeesMets",
             "Oakland Athletics":"San Francisco Bay Area GiantsAthletics",
             "San Francisco Giants":"San Francisco Bay Area GiantsAthletics",
             "New England Patriots":"Boston Patriots",
             "Los Angeles Chargers":"Los Angeles RamsChargers",
             "Los Angeles Rams":"Los Angeles RamsChargers",
             "Tennessee Titans":"Nashville Titans",
             "New York Giants":"New York City GiantsJets",
             "New York Jets":"New York City GiantsJets",
             "Oakland Raiders":"San Francisco Bay Area 49ersRaiders",
             "San Francisco 49ers":"San Francisco Bay Area 49ersRaiders",
             "New York Islanders":"New York City RangersIslandersDevils",
             "New Jersey Devils":"New York City RangersIslandersDevils"                          
}

#Manualling fixing names for teams
nhl_df_2018.replace({'team':names_fix}, inplace = True, regex=True)
nhl_df_2018.sort_values(by='team', inplace=True)
#Grouping same teams together and summing their wins and losses
nhl_df_2018 = nhl_df_2018.groupby('team').sum().reset_index()

#Caclulating win loss ratio
nhl_df_2018["W/L"] = nhl_df_2018["W"]/(nhl_df_2018["W"] + nhl_df_2018["L"])

#astype of all DFs
nhl_df_2018['team'] = nhl_df_2018['team'].astype(str)
nhl_df_2018["W/L"] = nhl_df_2018["W/L"].astype(float)

#reducing the size of the nhl dataframe
nhl_df_2018 = nhl_df_2018[['team','W/L']]
nhl_df_2018

'''
MERGE OPERATION
'''
#Joining NHL column with cities dataframe using left join to maintain
#the "source of authority" for the location of a given sports team
city_nhl = pd.merge(cities, nhl_df_2018, how='left', on='team')


city_nhl = city_nhl[['Metropolitan area','Population','W/L','team']]

#Change data to numeric type
city_nhl['Population'] = pd.to_numeric(city_nhl['Population'],errors='coerce')
city_nhl['W/L'] = pd.to_numeric(city_nhl['W/L'],errors='coerce')
#raise NotImplementedError()
    
population_by_region = city_nhl['Population'] # pass in metropolitan area population from cities
win_loss_by_region = city_nhl['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)

(0.012308996455744264, 0.9504308637909502)

In [319]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
pd.options.mode.chained_assignment = None  # default='warn'


def nhl_correlation():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    '''
    CITIES DATAFRAME CLEANUP AND PREPARATION
    '''
    #Removing trailing * and [xyz] from all columns
    cities.replace(to_replace = "\[.*\]$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—\s$", value ="", inplace = True, regex=True)

    #Replacing blank/epmty spaces with np.NaN
    cities.replace(to_replace = r'^\s*$', value =np.nan, inplace = True, regex=True)

    #Sorting for checking names across cities and sports dataframes
    cities.sort_values(by='Metropolitan area', inplace=True)

    #Dropping rows with np.NaN in the NHL column since we are analysis only NHL for this question
    cities.dropna(subset=['NHL'], inplace=True)

    #Joining NHL column with cities dataframe
    cities["team"] = cities[["Metropolitan area","NHL"]].apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)

    #Reducing the size of the cities dataframe after renaming Population (2016 est.)[8] column
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities = cities[['Metropolitan area','Population', 'team']]

    #astype of all DFs
    cities["Population"] = cities["Population"].astype(float)
    cities['team'] = cities['team'].astype(str)


    '''
    NHL DATAFRAME CLEANUP AND PREPARATION
    '''
    #Subsetting NHL dataframe tro 2018 only
    nhl_df_2018 = nhl_df[nhl_df["year"] == 2018]

    #Strips white spaces
    nhl_df_2018['team'] = nhl_df_2018['team'].str.strip()
    cities['team'] = cities['team'].str.strip()

    #Cleaning
    nhl_df_2018['team'].replace(to_replace = "[(].*", value ="", inplace = True, regex=True)
    nhl_df_2018['team'].replace(to_replace = "[\*]", value ="", inplace = True, regex=True)
    nhl_df_2018['team'].replace(to_replace = "[+]$", value ="", inplace = True, regex=True)

    #Making Win and Loss columns numeric and dropping strings or nan
    nhl_df_2018['W'] = pd.to_numeric(nhl_df_2018['W'],errors='coerce')
    nhl_df_2018.dropna(inplace=True)
    nhl_df_2018['L'] = pd.to_numeric(nhl_df_2018['L'],errors='coerce')

    #dictionary to fix names in the respective sports sheet
    names_fix = {"Minnesota":"Minneapolis–Saint Paul",
                "Florida":"Miami–Fort Lauderdale",
                "Dallas":"Dallas–Fort Worth",
                "Carolina Hurricanes":"Raleigh Hurricanes",
                "Colorado":"Denver",
                "Vegas":"Las Vegas",
                "Los Angeles Kings":"Los Angeles KingsDucks",
                "Anaheim Ducks":"Los Angeles KingsDucks",
                "New York Rangers":"New York City RangersIslandersDevils",
                "Arizona":"Phoenix",
                "San Jose":"San Francisco Bay Area",
                "Tampa Bay":"Tampa Bay Area",
                "Washington":"Washington, D.C.",
                "Los Angeles Clippers":"Los Angeles LakersClippers",
                "Los Angeles Lakers":"Los Angeles LakersClippers",
                "Golden State Warriors":"San Francisco Bay Area Warriors",
                "Miami":"Miami–Fort Lauderdale",
                "New York Knicks":"New York City KnicksNets",
                "Utah":"Salt Lake City",
                "Brooklyn Nets":"New York City KnicksNets",
                "Chicago Cubs":"Chicago CubsWhite Sox",
                "Chicago White Sox":"Chicago CubsWhite Sox",
                "Texas Rangers":"Dallas–Fort Worth Rangers",
                "Los Angeles Angels":"Los Angeles DodgersAngels",
                "Los Angeles Dodgers":"Los Angeles DodgersAngels",
                "New York Mets":"New York City YankeesMets",
                "New York Yankees":"New York City YankeesMets",
                "Oakland Athletics":"San Francisco Bay Area GiantsAthletics",
                "San Francisco Giants":"San Francisco Bay Area GiantsAthletics",
                "New England Patriots":"Boston Patriots",
                "Los Angeles Chargers":"Los Angeles RamsChargers",
                "Los Angeles Rams":"Los Angeles RamsChargers",
                "Tennessee Titans":"Nashville Titans",
                "New York Giants":"New York City GiantsJets",
                "New York Jets":"New York City GiantsJets",
                "Oakland Raiders":"San Francisco Bay Area 49ersRaiders",
                "San Francisco 49ers":"San Francisco Bay Area 49ersRaiders",
                "New York Islanders":"New York City RangersIslandersDevils",
                "New Jersey Devils":"New York City RangersIslandersDevils"                          
    }

    #Manualling fixing names for teams
    nhl_df_2018.replace({'team':names_fix}, inplace = True, regex=True)
    nhl_df_2018.sort_values(by='team', inplace=True)
    #Grouping same teams together and summing their wins and losses
    nhl_df_2018 = nhl_df_2018.groupby('team').sum().reset_index()

    #Caclulating win loss ratio
    nhl_df_2018["W/L"] = nhl_df_2018["W"]/(nhl_df_2018["W"] + nhl_df_2018["L"])

    #astype of all DFs
    nhl_df_2018['team'] = nhl_df_2018['team'].astype(str)
    nhl_df_2018["W/L"] = nhl_df_2018["W/L"].astype(float)

    #reducing the size of the nhl dataframe
    nhl_df_2018 = nhl_df_2018[['team','W/L']]


    '''
    MERGE OPERATION
    '''
    #Joining NHL column with cities dataframe using left join to maintain
    #the "source of authority" for the location of a given sports team
    city_nhl = pd.merge(cities, nhl_df_2018, how='left', on='team')


    city_nhl = city_nhl[['Metropolitan area','Population','W/L','team']]

    #Change data to numeric type
    city_nhl['Population'] = pd.to_numeric(city_nhl['Population'],errors='coerce')
    city_nhl['W/L'] = pd.to_numeric(city_nhl['W/L'],errors='coerce')
    #raise NotImplementedError()
        
    population_by_region = city_nhl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = city_nhl['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [320]:
nhl_correlation()

0.012308996455744264

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [321]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    '''
    CITIES DATAFRAME CLEANUP AND PREPARATION
    '''
    #Removing trailing * and [xyz] from all columns
    cities.replace(to_replace = "\[.*\]$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—\s$", value ="", inplace = True, regex=True)

    #Replacing blank/epmty spaces with np.NaN
    cities.replace(to_replace = r'^\s*$', value =np.nan, inplace = True, regex=True)

    #Sorting for checking names across cities and sports dataframes
    cities.sort_values(by='Metropolitan area', inplace=True)

    #Dropping rows with np.NaN in the NBA column since we are analysis only NBA for this question
    cities.dropna(subset=['NBA'], inplace=True)

    #Joining NBA column with cities dataframe
    cities["team"] = cities[["Metropolitan area","NBA"]].apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)

    #Reducing the size of the cities dataframe after renaming Population (2016 est.)[8] column
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities = cities[['Metropolitan area','Population', 'team']]
    
    #astype of all DFs
    cities["Population"] = cities["Population"].astype(float)
    cities['team'] = cities['team'].astype(str)

    '''
    NBA DATAFRAME CLEANUP AND PREPARATION
    '''
    #Subsetting NBA dataframe tro 2018 only
    nba_df_2018 = nba_df[nba_df["year"] == 2018]

    #Strips white spaces
    nba_df_2018['team'] = nba_df_2018['team'].str.strip()
    cities['team'] = cities['team'].str.strip()

    #Cleaning
    nba_df_2018['team'].replace(to_replace = "[(].*", value ="", inplace = True, regex=True)
    nba_df_2018['team'].replace(to_replace = "[\*]", value ="", inplace = True, regex=True)

    #Making Win and Loss columns numeric and dropping strings or nan
    nba_df_2018['W'] = pd.to_numeric(nba_df_2018['W'],errors='coerce')
    nba_df_2018.dropna(inplace=True)
    nba_df_2018['L'] = pd.to_numeric(nba_df_2018['L'],errors='coerce')

    #dictionary to fix names in the respective sports sheet
    names_fix = {"Minnesota":"Minneapolis–Saint Paul",
                "Florida":"Miami–Fort Lauderdale",
                "Dallas":"Dallas–Fort Worth",
                "Colorado":"Denver",
                "Vegas":"Las Vegas",
                "Los Angeles Kings":"Los Angeles KingsDucks",
                "New York Rangers":"New York City RangersIslandersDevils",
                "Arizona":"Phoenix",
                "Carolina":"Raleigh",
                "San Jose":"San Francisco Bay Area",
                "Tampa Bay":"Tampa Bay Area",
                "Washington":"Washington, D.C.",
                "Indiana":"Indianapolis",
                "Los Angeles Clippers":"Los Angeles LakersClippers",
                "Los Angeles Lakers":"Los Angeles LakersClippers",
                "Golden State Warriors":"San Francisco Bay Area Warriors",
                "Miami":"Miami–Fort Lauderdale",
                "New York Knicks":"New York City KnicksNets",
                "Utah":"Salt Lake City",
                "Brooklyn Nets":"New York City KnicksNets",
                
    }

    #Manualling fixing names for teams
    nba_df_2018.replace({'team':names_fix}, inplace = True, regex=True)
    nba_df_2018.sort_values(by='team', inplace=True)


    #astype of all DFs
    nba_df_2018['team'] = nba_df_2018['team'].astype(str)
    nba_df_2018["W/L%"] = nba_df_2018["W/L%"].astype(float)


    #Grouping same teams together and summing their wins and losses
    nba_df_2018 = nba_df_2018.groupby('team').sum().reset_index()

    #Caclulating win loss ratio
    nba_df_2018["W/L"] = nba_df_2018["W"]/(nba_df_2018["W"] + nba_df_2018["L"])
    #astype of all DFs
    nba_df_2018["W/L"] = nba_df_2018["W/L"].astype(float)    

    #reducing the size of the nba dataframe
    nba_df_2018 = nba_df_2018[['team','W/L%','W/L']]

    #Strips white spaces on team column since merger was returing NaN values
    nba_df_2018['team'] = nba_df_2018['team'].str.strip()
    cities['team'] = cities['team'].str.strip()


    #Joining NBA column with cities dataframe using left join to maintain
    #the "source of authority" for the location of a given sports team
    city_nba = pd.merge(cities, nba_df_2018, how='left', on='team')
    city_nba = city_nba[['Metropolitan area','Population','W/L','team']]

    #Change data to numeric type
    city_nba['Population'] = pd.to_numeric(city_nba['Population'],errors='coerce')
    city_nba['W/L'] = pd.to_numeric(city_nba['W/L'],errors='coerce')
    #raise NotImplementedError()
        
    population_by_region = city_nba['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = city_nba['W/L'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [322]:
nba_correlation()

-0.17657160252844611

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [323]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    '''
    CITIES DATAFRAME CLEANUP AND PREPARATION
    '''
    #Removing trailing * and [xyz] from all columns
    cities.replace(to_replace = "\[.*\]$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—\s$", value ="", inplace = True, regex=True)

    #Replacing blank/epmty spaces with np.NaN
    cities.replace(to_replace = r'^\s*$', value =np.nan, inplace = True, regex=True)

    #Sorting for checking names across cities and sports dataframes
    cities.sort_values(by='Metropolitan area', inplace=True)

    #Dropping rows with np.NaN in the MLB column since we are analysis only MLB for this question
    cities.dropna(subset=['MLB'], inplace=True)

    #Joining MLB column with cities dataframe
    cities["team"] = cities[["Metropolitan area","MLB"]].apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)

    #Reducing the size of the cities dataframe after renaming Population (2016 est.)[8] column
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities = cities[['Metropolitan area','Population', 'team']]

    #astype of all DFs
    cities["Population"] = cities["Population"].astype(int)
    cities['team'] = cities['team'].astype(str)

    '''
    MLB DATAFRAME CLEANUP AND PREPARATION
    '''
    #Subsetting MLB dataframe tro 2018 only
    mlb_df_2018 = mlb_df[mlb_df["year"] == 2018]

    #Cleaning
    mlb_df_2018['team'].replace(to_replace = "[(].*", value ="", inplace = True, regex=True)
    mlb_df_2018['team'].replace(to_replace = "[\*]", value ="", inplace = True, regex=True)

    #Making Win and Loss columns numeric and dropping strings or nan
    mlb_df_2018['W'] = pd.to_numeric(mlb_df_2018['W'],errors='coerce')
    mlb_df_2018.dropna(inplace=True)
    mlb_df_2018['L'] = pd.to_numeric(mlb_df_2018['L'],errors='coerce')

    #dictionary to fix names in the respective sports sheet
    names_fix = {"Minnesota":"Minneapolis–Saint Paul",
                "Florida":"Miami–Fort Lauderdale",
                "Dallas":"Dallas–Fort Worth",
                "Colorado":"Denver",
                "Vegas":"Las Vegas",
                "Los Angeles Kings":"Los Angeles KingsDucks",
                "New York Rangers":"New York City RangersIslandersDevils",
                "Arizona":"Phoenix",
                "Carolina":"Raleigh",
                "San Jose":"San Francisco Bay Area",
                "Tampa Bay":"Tampa Bay Area",
                "Washington":"Washington, D.C.",
                "Indiana":"Indianapolis",
                "Los Angeles Clippers":"Los Angeles LakersClippers",
                "Los Angeles Lakers":"Los Angeles LakersClippers",
                "Golden State Warriors":"San Francisco Bay Area Warriors",
                "Miami":"Miami–Fort Lauderdale",
                "New York Knicks":"New York City KnicksNets",
                "Utah":"Salt Lake City",
                "Brooklyn Nets":"New York City KnicksNets",
                "Chicago Cubs":"Chicago CubsWhite Sox",
                "Chicago White Sox":"Chicago CubsWhite Sox",
                "Texas Rangers":"Dallas–Fort Worth Rangers",
                "Los Angeles Angels":"Los Angeles DodgersAngels",
                "Los Angeles Dodgers":"Los Angeles DodgersAngels",
                "New York Mets":"New York City YankeesMets",
                "New York Yankees":"New York City YankeesMets",
                "Oakland Athletics":"San Francisco Bay Area GiantsAthletics",
                "San Francisco Giants":"San Francisco Bay Area GiantsAthletics",
                
    }

    #Manualling fixing names for teams
    mlb_df_2018.replace({'team':names_fix}, inplace = True, regex=True)
    mlb_df_2018.sort_values(by='team', inplace=True)



    #astype of all DFs
    mlb_df_2018['team'] = mlb_df_2018['team'].astype(str)
    #mlb_df_2018["W/L%"] = mlb_df_2018["W/L%"].astype(float)


    #Grouping same teams together and summing their wins and losses
    mlb_df_2018 = mlb_df_2018.groupby('team').sum().reset_index()

    #Caclulating win loss ratio
    mlb_df_2018["W/L"] = mlb_df_2018["W"]/(mlb_df_2018["W"] + mlb_df_2018["L"])
    mlb_df_2018["W/L"] = mlb_df_2018["W/L"].astype(float)

    #reducing the size of the mlb dataframe
    mlb_df_2018 = mlb_df_2018[['team','W/L']]


    '''
    MERGE OPERATION
    '''
    #Strips white spaces
    mlb_df_2018['team'] = mlb_df_2018['team'].str.strip()
    cities['team'] = cities['team'].str.strip()

    #Joining MLB column with cities dataframe using left join to maintain
    #the "source of authority" for the location of a given sports team
    city_mlb = pd.merge(cities, mlb_df_2018, how='left', on='team')

    city_mlb = city_mlb[['Metropolitan area','Population','W/L','team']]

    #Change data to numeric type
    city_mlb['Population'] = pd.to_numeric(city_mlb['Population'],errors='coerce')
    city_mlb['W/L'] = pd.to_numeric(city_mlb['W/L'],errors='coerce')
    #raise NotImplementedError()
        
    population_by_region = city_mlb['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = city_mlb['W/L'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [324]:
mlb_correlation()

0.15052304487104853

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [325]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    '''
    CITIES DATAFRAME CLEANUP AND PREPARATION
    '''
    #Removing trailing * and [xyz] from all columns
    cities.replace(to_replace = "\[.*\]$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—$", value ="", inplace = True, regex=True)
    cities.replace(to_replace = ".*—\s$", value ="", inplace = True, regex=True)

    #Replacing blank/epmty spaces with np.NaN
    cities.replace(to_replace = r'^\s*$', value =np.nan, inplace = True, regex=True)

    #Sorting for checking names across cities and sports dataframes
    cities.sort_values(by='Metropolitan area', inplace=True)

    #Dropping rows with np.NaN in the NFL column since we are analysis only NFL for this question
    cities.dropna(subset=['NFL'], inplace=True)

    #Joining NFL column with cities dataframe
    cities["team"] = cities[["Metropolitan area","NFL"]].apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)

    #Reducing the size of the cities dataframe after renaming Population (2016 est.)[8] column
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities = cities[['Metropolitan area','Population', 'team']]

    #astype of all DFs
    cities["Population"] = cities["Population"].astype(int)
    cities['team'] = cities['team'].astype(str)

    '''
    NFL DATAFRAME CLEANUP AND PREPARATION
    '''
    #Subsetting NFL dataframe tro 2018 only
    nfl_df_2018 = nfl_df[nfl_df["year"] == 2018]

    #Strips white spaces
    nfl_df_2018['team'] = nfl_df_2018['team'].str.strip()
    cities['team'] = cities['team'].str.strip()

    #Cleaning
    nfl_df_2018['team'].replace(to_replace = "[(].*", value ="", inplace = True, regex=True)
    nfl_df_2018['team'].replace(to_replace = "[\*]", value ="", inplace = True, regex=True)
    nfl_df_2018['team'].replace(to_replace = "[+]$", value ="", inplace = True, regex=True)

    #Making Win and Loss columns numeric and dropping strings or nan
    nfl_df_2018['W'] = pd.to_numeric(nfl_df_2018['W'],errors='coerce')
    nfl_df_2018.dropna(inplace=True)
    nfl_df_2018['L'] = pd.to_numeric(nfl_df_2018['L'],errors='coerce')

    #dictionary to fix names in the respective sports sheet
    names_fix = {"Minnesota":"Minneapolis–Saint Paul",
                "Florida":"Miami–Fort Lauderdale",
                "Dallas":"Dallas–Fort Worth",
                "Colorado":"Denver",
                "Vegas":"Las Vegas",
                "Los Angeles Kings":"Los Angeles KingsDucks",
                "New York Rangers":"New York City RangersIslandersDevils",
                "Arizona":"Phoenix",
                "San Jose":"San Francisco Bay Area",
                "Tampa Bay":"Tampa Bay Area",
                "Washington":"Washington, D.C.",
                "Los Angeles Clippers":"Los Angeles LakersClippers",
                "Los Angeles Lakers":"Los Angeles LakersClippers",
                "Golden State Warriors":"San Francisco Bay Area Warriors",
                "Miami":"Miami–Fort Lauderdale",
                "New York Knicks":"New York City KnicksNets",
                "Utah":"Salt Lake City",
                "Brooklyn Nets":"New York City KnicksNets",
                "Chicago Cubs":"Chicago CubsWhite Sox",
                "Chicago White Sox":"Chicago CubsWhite Sox",
                "Texas Rangers":"Dallas–Fort Worth Rangers",
                "Los Angeles Angels":"Los Angeles DodgersAngels",
                "Los Angeles Dodgers":"Los Angeles DodgersAngels",
                "New York Mets":"New York City YankeesMets",
                "New York Yankees":"New York City YankeesMets",
                "Oakland Athletics":"San Francisco Bay Area GiantsAthletics",
                "San Francisco Giants":"San Francisco Bay Area GiantsAthletics",
                "New England Patriots":"Boston Patriots",
                "Carolina Panthers":"Charlotte Panthers",
                "Los Angeles Chargers":"Los Angeles RamsChargers",
                "Los Angeles Rams":"Los Angeles RamsChargers",
                "Tennessee Titans":"Nashville Titans",
                "New York Giants":"New York City GiantsJets",
                "New York Jets":"New York City GiantsJets",
                "Oakland Raiders":"San Francisco Bay Area 49ersRaiders",
                "San Francisco 49ers":"San Francisco Bay Area 49ersRaiders",                          
    }

    #Manualling fixing names for teams
    nfl_df_2018.replace({'team':names_fix}, inplace = True, regex=True)
    nfl_df_2018.sort_values(by='team', inplace=True)



    #astype of all DFs
    nfl_df_2018['team'] = nfl_df_2018['team'].astype(str)
    #nfl_df_2018["W/L%"] = nfl_df_2018["W/L%"].astype(float)


    #Grouping same teams together and summing their wins and losses
    nfl_df_2018 = nfl_df_2018.groupby('team').sum().reset_index()


    #Caclulating win loss ratio
    nfl_df_2018["W/L"] = nfl_df_2018["W"]/(nfl_df_2018["W"] + nfl_df_2018["L"])
    nfl_df_2018["W/L"] = nfl_df_2018["W/L"].astype(float)
    
    #reducing the size of the nfl dataframe
    nfl_df_2018 = nfl_df_2018[['team','W/L']]
    
    '''
    MERGE OPERATION
    '''
    #Joining NFL column with cities dataframe using left join to maintain
    #the "source of authority" for the location of a given sports team
    city_nfl = pd.merge(cities, nfl_df_2018, how='left', on='team')

    city_nfl = city_nfl[['Metropolitan area','Population','W/L','team']]

    #Change data to numeric type
    city_nfl['Population'] = pd.to_numeric(city_nfl['Population'],errors='coerce')
    city_nfl['W/L'] = pd.to_numeric(city_nfl['W/L'],errors='coerce')
    #raise NotImplementedError()
        
    population_by_region = city_nfl['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = city_nfl['W/L'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [326]:
nfl_correlation()

0.004922112149349428

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values